<a href="https://colab.research.google.com/github/mingmingbupt/tensorflow/blob/master/tf_data_basic_api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
import os
import sys
import time
import tensorflow as tf

from tensorflow import keras

print(tf.__version__)
print(sys.version_info)
for module in mpl, np, pd, sklearn, tf, keras:
    print(module.__name__, module.__version__)

2.2.0-rc2
sys.version_info(major=3, minor=6, micro=9, releaselevel='final', serial=0)
matplotlib 3.2.1
numpy 1.18.2
pandas 1.0.3
sklearn 0.22.2.post1
tensorflow 2.2.0-rc2
tensorflow.keras 2.3.0-tf


In [10]:
dataset = tf.data.Dataset.from_tensor_slices(np.arange(10)) #从内存中构建数据集
#他里面参数呢，可以是一个普通的列表，可以是numpy的数组，甚至可以是一个字典，这里我们用numpy的数组对他进行初始化
#大小为长度为10的一个一维向量，这样我们就初始化好了一个dataset,打印下
print(dataset)

<TensorSliceDataset shapes: (), types: tf.int64>


In [11]:
for item in dataset: #对于一个dataset 可以做些什么，首先可以遍历
    #我们用一个for循环对dataset进行遍历，获取dataset中每一个数据，发现每一个数据就是一个tensor，对应的就是0-9这10个数字
    print(item)

tf.Tensor(0, shape=(), dtype=int64)
tf.Tensor(1, shape=(), dtype=int64)
tf.Tensor(2, shape=(), dtype=int64)
tf.Tensor(3, shape=(), dtype=int64)
tf.Tensor(4, shape=(), dtype=int64)
tf.Tensor(5, shape=(), dtype=int64)
tf.Tensor(6, shape=(), dtype=int64)
tf.Tensor(7, shape=(), dtype=int64)
tf.Tensor(8, shape=(), dtype=int64)
tf.Tensor(9, shape=(), dtype=int64)


In [12]:
# 针对一个机器学习，我们使用一个dataset的时候，最常用的几个操作是什么
# 1. repeat epoch 对一个dataset进行重复读取。遍历一次数据集，就称为一次epoch.数据集遍历多少次呢，就称我这个训练集运行了多少个epoch
#   之前我们调用keras,调用.fit函数，有进度条在哪里走，每进行一个epoch他就换新的一行
# 2. get batch 第二个常见的操作，就是get batch.在之前的例子中呢，需要指定一个batch_size,这个batch_size就是我在每一次训练我的网络参数的时候
#   我不是把所有数据都输入进去，而是我从整个数据集中取一小部分出来，然后把这个数据再丢给训练程序，再去计算导数，更新参数

# dataset想要实现上面两个功能，非常简单，只需要对dataset进行变换，产生新的dataset.
# 首先呢，叫repeat repeat(3)就是0-10会遍历3次，然后我们再调用batch函数，这里batch_size=7,就是每次返回7个数据
# 我先调用repeat，直接在返回结果上调用batch函数
# 这个实现机制就在于我的dataset数据调用repeat函数后呢，会产生新的dataset.同理呢，我再调用batch函数呢，他还会产生一个新的dataset
# 所以我的返回结果就是一个新的dataset
dataset = dataset.repeat(3).batch(7)
for item in dataset:
    print(item)

tf.Tensor([0 1 2 3 4 5 6], shape=(7,), dtype=int64)
tf.Tensor([7 8 9 0 1 2 3], shape=(7,), dtype=int64)
tf.Tensor([4 5 6 7 8 9 0], shape=(7,), dtype=int64)
tf.Tensor([1 2 3 4 5 6 7], shape=(7,), dtype=int64)
tf.Tensor([8 9], shape=(2,), dtype=int64)


In [13]:
# interleave: 这个函数用来对dataset中每一个元素做处理，每个元素做完处理会产生新的结果，然后interleave会把新的结果再合并起来
#        形成一个新的数据集
# interleave 一个常见的case是我现有的dataset里面存储的是一系列文件的文件名，我用interleave去做一个变换，这个变换是我遍历这个文件名
#       数据集里面的所有的元素，也就是文件名，然后呢把文件名对应的文件内容给读取出来，这样每个文件名都形成了一个新的数据集
#       然后interleave再把这些新的数据集再合并起来，成为一个总的大数据集，这是一个常见的case
# case: 文件dataset -> 具体数据集

# 这里举一个其他的例子，在interleave中有三个参数
# 第一个参数是map_fn 也就是说我去做一个什么样的变换
# 第二个参数是一个叫做cycle_length 也就是他的并行程度，就是说我并行的同时去这里这个dataset的多少个元素
# 第三个参数是block_length, 我从上面的变换结果里每次取多少个元素出来
dataset2 = dataset.interleave(
    lambda v: tf.data.Dataset.from_tensor_slices(v), # map_fn，使用一个lambda表达式来定义一个新的匿名函数，我每次都从dataset里每次都取出一个v
    # 然后我把这个v封装成一个新的数据集
    cycle_length = 5, # cycle_length, 在结果里是看不出来的，因为他是并行程度的一个衡量
    block_length = 5, # block_length
)

for item in dataset:
    print(item)

for item in dataset2: #遍历dataset2，然后我们就看到了
    print(item)
# 结果里面每个都是一个tensor,每个tensor里面只有一个元素
# 效果就是我每次都是从其中某个元素的结果里面呢取一小部分出来，生成出来
# 再从另外一个元素结果里面取出来某些值，然后再去填充我这个数据集
# 所以说呢，他就达到了一种均匀混合的一种效果，这是block_length的一个效果

tf.Tensor([0 1 2 3 4 5 6], shape=(7,), dtype=int64)
tf.Tensor([7 8 9 0 1 2 3], shape=(7,), dtype=int64)
tf.Tensor([4 5 6 7 8 9 0], shape=(7,), dtype=int64)
tf.Tensor([1 2 3 4 5 6 7], shape=(7,), dtype=int64)
tf.Tensor([8 9], shape=(2,), dtype=int64)
tf.Tensor(0, shape=(), dtype=int64)
tf.Tensor(1, shape=(), dtype=int64)
tf.Tensor(2, shape=(), dtype=int64)
tf.Tensor(3, shape=(), dtype=int64)
tf.Tensor(4, shape=(), dtype=int64)
tf.Tensor(7, shape=(), dtype=int64)
tf.Tensor(8, shape=(), dtype=int64)
tf.Tensor(9, shape=(), dtype=int64)
tf.Tensor(0, shape=(), dtype=int64)
tf.Tensor(1, shape=(), dtype=int64)
tf.Tensor(4, shape=(), dtype=int64)
tf.Tensor(5, shape=(), dtype=int64)
tf.Tensor(6, shape=(), dtype=int64)
tf.Tensor(7, shape=(), dtype=int64)
tf.Tensor(8, shape=(), dtype=int64)
tf.Tensor(1, shape=(), dtype=int64)
tf.Tensor(2, shape=(), dtype=int64)
tf.Tensor(3, shape=(), dtype=int64)
tf.Tensor(4, shape=(), dtype=int64)
tf.Tensor(5, shape=(), dtype=int64)
tf.Tensor(8, shape=(), dtype=i

In [14]:
x = np.array([[1, 2], [3, 4], [5, 6]])
y = np.array(['cat', 'dog', 'fox']) #可能对应二维矩阵的类型，比如label
#x和y的第一个维度一定是相等的。才能够正常工作，这里x长度3,y也是3
#然后我用dataset进行构建，我的输入是一个元组(x,y)
#from_tensor_slices除了支持numpy向量以外，还支持python中一些原有的数据
#比如说元组和字典
#这里输入的是一个元祖（x,y）
dataset3 = tf.data.Dataset.from_tensor_slices((x, y))
print(dataset3)

for item_x, item_y in dataset3:
    print(item_x.numpy(), item_y.numpy())
    print(item_x, item_y)
#item_x, item_y 都是tf.tensor
#item_x, item_y 都是tf.tensor

<TensorSliceDataset shapes: ((2,), ()), types: (tf.int64, tf.string)>
[1 2] b'cat'
tf.Tensor([1 2], shape=(2,), dtype=int64) tf.Tensor(b'cat', shape=(), dtype=string)
[3 4] b'dog'
tf.Tensor([3 4], shape=(2,), dtype=int64) tf.Tensor(b'dog', shape=(), dtype=string)
[5 6] b'fox'
tf.Tensor([5 6], shape=(2,), dtype=int64) tf.Tensor(b'fox', shape=(), dtype=string)


In [16]:
#from_tensor_slices也支持字典
#
dataset4 = tf.data.Dataset.from_tensor_slices({"feature": x, "label": y})
for item in dataset4:
    print(item)
    print(item["feature"].numpy(), item["label"].numpy())
    print(item["feature"], item["label"])

{'feature': <tf.Tensor: shape=(2,), dtype=int64, numpy=array([1, 2])>, 'label': <tf.Tensor: shape=(), dtype=string, numpy=b'cat'>}
[1 2] b'cat'
tf.Tensor([1 2], shape=(2,), dtype=int64) tf.Tensor(b'cat', shape=(), dtype=string)
{'feature': <tf.Tensor: shape=(2,), dtype=int64, numpy=array([3, 4])>, 'label': <tf.Tensor: shape=(), dtype=string, numpy=b'dog'>}
[3 4] b'dog'
tf.Tensor([3 4], shape=(2,), dtype=int64) tf.Tensor(b'dog', shape=(), dtype=string)
{'feature': <tf.Tensor: shape=(2,), dtype=int64, numpy=array([5, 6])>, 'label': <tf.Tensor: shape=(), dtype=string, numpy=b'fox'>}
[5 6] b'fox'
tf.Tensor([5 6], shape=(2,), dtype=int64) tf.Tensor(b'fox', shape=(), dtype=string)
